In [1]:
# 문제 정의 : 전국 병원정보 데이터, 전국 약국 정보 데이터, 전국 지역별 연령별 총 인구수 데이터를 통해 의료서비스 밀집 지역과 아닌 지역을 도출해
# 고령층이 많이 거주하는 지역에 의료 시설이 충분히 제공되고 있는지 확인하고자 합니다.

# 예상 결과 : 전라도와 강원도 지역이 인구 밀도와 인프라 밀집도가 낮아 의료 수요가 부족할 것으로 예상됩니다.

In [2]:
# 필요한 라이브러리 불러오기

import pandas as pd #데이터 프레임을 다루는 라이브러리
import numpy as np #수치계산을 하기 위한 라이브러리
import matplotlib.pyplot as plt #데이터 시각화를 위한 라이브러리
from sklearn.cluster import KMeans #kmeans 클러스터링 알고리즘을 위한 라이브러리
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [ ]:
# 데이터 로드
# 데이터 파일의 경로를 설정해주세요.

file_path1 = "1.병원정보서비스 2024.3.csv"
file_path2 = "2.약국정보서비스 2024.3.csv"

data = pd.read_csv(data_path1)
